In [193]:
import pandas as pd

!pip install sqlparse --user --upgrade
!pip install sqlglot --user --upgrade

import sqlglot
import sqlglot.expressions as exp
from sqlglot import parse_one
from sqlglot.optimizer import optimize
from sqlglot import optimizer
from sqlglot.errors import OptimizeError
from sqlglot import lineage

  Obtaining dependency information for sqlglot from https://files.pythonhosted.org/packages/6c/6e/ee658ca20ea29804ea7bc226df2381bbb95dac6a3735fb6218d1657e4d43/sqlglot-25.15.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/400.5 kB ? eta -:--:--
   ----------- ---------------------------- 112.6/400.5 kB 2.2 MB/s eta 0:00:01
   ---------------- ----------------------- 163.8/400.5 kB 2.4 MB/s eta 0:00:01
   ------------------------------- -------- 317.4/400.5 kB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 400.5/400.5 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: sqlglot
    Found existing installation: sqlglot 25.14.0
    Uninstalling sqlglot-25.14.0:
      Successfully uninstalled sqlglot-25.14.0


In [194]:
with open('stored procedure/sp_otp_shipment_level.txt', 'r') as f:
    #remove illegal words
    text = f.read().replace('~', '!=')
    lines = text.split(';')

In [195]:
sql_2_b_processed = []
fail_2_processed = []
stored_procedure_called = []

In [196]:
for sql in lines:
    
    if ('insert' in sql.lower() or 'update' in sql.lower()): #grab all building block to build the database
        
        sql_2_b_processed.append(sql)
        
    elif ('call' in sql.lower()): #to grap all procedure called within procedure
        
        stored_procedure_called.append(sql)
        
    else: 
    
        fail_2_processed.append(sql)
        
len(sql_2_b_processed)

38

In [197]:
#prepocessing
#change update and insert to respective select statement
#addtionally if it is insert: prepare list of output table columns

sql = sql_2_b_processed[0]

token = ''

if('insert' in sql.lower()):
    
    token = 'insert'
    
    for expression in sqlglot.parse(sql):
        insert_table_name = str(expression.args['this'].this)
        columns = []
        for column in expression.args['this'].expressions:
            columns.append(column.this)
        expression_ = str(expression.expression)
    
    #recompose the list of column into a single string: REAL Columns for the datamart
    insert_columns = ', '.join(columns)[:-2]
    
    sql = expression_
    #print(sql)
    
elif('update' in sql.lower()):
    
    token = 'update'
    
    for expression in sqlglot.parse(sql):
        #print(expression.args)
        table_name = str(expression.args['this'])
        field_value_name = expression.expressions
    
        fieldname = []
        value = []
    
        for pair in field_value_name:
            fieldname.append(str(pair.this))
            value.append(str(pair.expression))
        
        from_ = str(expression.args['from'])
        where = str(expression.args['where'])

        sql_ = ''
    
        for value_, fieldname_ in zip(value, fieldname):
    
            sql_ = sql_ + ' ' + value_ + ' AS ' + fieldname_ + ','
    
        sql_ = sql_[:-1]

        sql_ = 'SELECT ' + sql_ + ' FROM ' + table_name + ' JOIN ' + from_.replace('FROM', ' ') + ' ON (' + where.replace('WHERE', ' ') + ')' 
        sql = sql_
    
        update_column = fieldname
        all_column = update_column

else:
    print('no action.')
    
#print(sql)

Format argument unsupported for TO_CHAR/TO_VARCHAR function


In [257]:
#helper functions
#find all alias from cte and related table and schema
def obtain_table_alias_type(sql):
    
    for alias in parse_one(sql, dialect="redshift").find_all(exp.TableAlias):
        
        print(f"alias => {alias.this.this} | alias_table_type => {type(alias.parent_select)}" )
        
        break
        
#find schema by name
def find_schema_by_table_name(table_name):
    
    for table in parse_one(sql, dialect="redshift").find_all(exp.Table):
        
        if (table_name == table.name):
            return str(table.args['db'])
            
def obtain_list_column_table(sql):
    
    column_l = []
    table_l = []
    alias_l = []
    
    for column in parse_one(sql, dialect="redshift").find_all(exp.Column):
        
        column_l.append(column.name)
        table_l.append(column.table)
        #print(column.key)
        #print(f"Column => {column.name} | DB => {column.table}" )

    return column_l, table_l

#find all alias from cte and related table and schema
def obtain_list_table_alias(sql):
    
    unalias_name = []
    alias_l = []
    related_table_l = []
    alias_type_ = []
    schema_l = []
    
    for alias in parse_one(sql, dialect="redshift").find_all(exp.TableAlias):
        
        table_t = []
        #if it is cte
        if (alias.parent.name == ''):
            alias_type_.append(type(alias.parent.args['this']))
            alias_l.append(alias.this.this)
            column_t, table_t = obtain_list_column_table(str(alias.parent))
            related_table_l.append(list(dict.fromkeys(table_t)))
            unalias_name.append(alias.parent.name)
            
            schema = []
            
            for table in list(dict.fromkeys(table_t)):
                
                schema.append(find_schema_by_table_name(table))
                
            schema_l.append(schema)
            
        #if it is normal table
        else:
            alias_type_.append('')
            related_table_l.append('')
            alias_l.append(alias.this.this)
            unalias_name.append(alias.parent.name)
            
            schema_l.append(find_schema_by_table_name(alias.parent.name))
        
        #print(f"Column => {alias_l} | DB => {related_table_l}" )
        
    return alias_l, related_table_l, alias_type_, unalias_name, schema_l

def find_column_originated(sql):

    column_l = []
    table_l = []
    originated_l = []
    
    group_column_l = []
    group_table_l = []
    group_originated_l = []
    #print(sql)
    
    #need to use parse instead of parse_one
    for expression in sqlglot.parse(sql):
        #print(expression.args)
        
        keysList = list(expression.args.keys())
        #print(keysList)
    
        for key in keysList:
            if expression.args[key] != None:
                
                #find all expression/join key
                if type(expression.args[key]) is list:
                    for objecto in expression.args[key]:
                        for column in objecto.find_all(exp.Column):
                            #print(f"Column => {column.name} | type => {key}" )
                            column_l.append(column.name)
                            table_l.append(column.table)
                            originated_l.append(key)
                else:
                    #list all group by items
                    if key == 'group':
                        for groupby_clause in expression.args[key].expressions:
                            if (type(groupby_clause)==sqlglot.expressions.Column):
                                group_column_l.append(groupby_clause.this)
                                group_table_l.append(groupby_clause.table)
                            else:
                                group_column_l.append(groupby_clause)
                                group_table_l.append(None)
                            group_originated_l.append('group by')
    
    #combine into one single dataframe
    dict = {'output_column': column_l, 'table': table_l, 'action': originated_l} 
    df_relationship = pd.DataFrame(dict)
    dict = {'output_column': group_column_l, 'table': group_table_l, 'action': group_originated_l}
    df_group = pd.DataFrame(dict)
    df_relationship = df_relationship.append(df_group)
    
    return df_relationship

#just in case if there is union
def deep_find_column_originated(sql, table_name):
    
    sqlglot_ = parse_one(sql)
    if (type(sqlglot_)==sqlglot.expressions.Union):
        df_union_1 = find_column_originated(str(sqlglot_.args['this']))
        df_union_2 = find_column_originated(str(sqlglot_.args['expression']))
        
        df_union_1['union'] = ' left union'
        df_union_2['union'] = ' right union'
        
        df_ = pd.concat([df_union_1, df_union_2], ignore_index=True)
        
    else:
        df_ = find_column_originated(sql)
        df_['union'] = None
        
    df_['destined_table'] = table_name
    return df_ 


In [258]:
#find the position of the column in the sql, whether it is in select/update/insert or join clause or group by
df_relation = deep_find_column_originated(sql, insert_table_name)

for subquery in parse_one(sql).find_all(exp.Subquery):
    
    df_subquery = deep_find_column_originated(str(subquery.args['this']), subquery.alias)
    df_relation = pd.concat([df_relation, df_subquery], ignore_index=True)

df_relation

groupby_df = df_relation.loc[df_relation['action']=='group by']
column_df = df_relation.loc[df_relation['action']=='expressions']
join_df = df_relation.loc[df_relation['action']=='joins']

join_df

C:\Users\ELAM2\AppData\Local\Temp\ipykernel_32324\4141691527.py:121: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_relationship = df_relationship.append(df_group)
C:\Users\ELAM2\AppData\Local\Temp\ipykernel_32324\4141691527.py:121: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_relationship = df_relationship.append(df_group)
C:\Users\ELAM2\AppData\Local\Temp\ipykernel_32324\4141691527.py:121: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_relationship = df_relationship.append(df_group)


,output_column,table,action,union,destined_table
95,changeoperation,vendor,joins,None,qma_datamart.otp_shipment_level
96,rda_code,z,joins,None,qma_datamart.otp_shipment_level
97,fty_code,vendor,joins,None,qma_datamart.otp_shipment_level
98,src_sys,z,joins,None,qma_datamart.otp_shipment_level
99,src_sys,vendor,joins,None,qma_datamart.otp_shipment_level
...,...,...,...,...,...
487,scale_desc,dim_size_scale,joins,right union,z
488,size,fact_po_dtl,joins,right union,z
489,scale_code,dim_size_scale,joins,right union,z
490,src_sys,fact_po_dtl,joins,right union,z


In [8]:
#preprocessing: for Insert
#find all output columns, columns, table, and schema
#1. find all output columns
column_list = []
table_list = []
alias_list = []
is_calculated = []

for select in parse_one(sql).find_all(exp.Select):
    for projection in select.expressions:
        temp = projection['this'].args
        if (isinstance(temp['this'], sqlglot.expressions.Alias)):
            
            if (isinstance(temp['this'].this, sqlglot.expressions.Column)):
                #column
                column_list.append(temp['this'].this.this.this)
                #table
                table_list.append(temp['this'].this.table)
                #alias
                alias_list.append(temp['this'].alias)
            else:
                column_list.append('')
                table_list.append('')
                alias_list.append('')
            
        elif (isinstance(temp['this'], sqlglot.expressions.Column)):
            #column
            column_list.append(temp['this'].this.this)
            #table
            table_list.append(temp['this'].table)
            alias_list.append('')
           
        else:
            column_list.append('')
            table_list.append('')
            alias_list.append('')
    break

# Convert to DataFrame
dict = {'output_column': all_column, 'first_level_column': column_list, 'first_level_table': table_list, 'first_level_alias': alias_list} 
   
df = pd.DataFrame(dict)
df.loc[df['first_level_column']!='', 'first_level_relationship'] = 'extract'
df.loc[df['first_level_column']=='', 'first_level_relationship'] = 'calculated'

df[['output_column', 'first_level_relationship']]

,output_column,first_level_relationship
0,country_of_origin,extract
1,po_cut,extract
2,style,extract
3,color,extract
4,style_description,extract
...,...,...
68,delivery_mode_shipment_load_type,extract
69,pts_issue_date,calculated
70,order_amount_local_currency,calculated
71,shipped_amt_local_currency,calculated


In [46]:
column = 'vendor_ffc'

print(len(list(lineage.lineage(column, sql).walk())))

source_list = []
expression_list = []
column_list = []
alias_list = []
reference_list = []

for node in lineage.lineage(column, sql).walk():
    
    #field name
    node_ = str(node.name)
    
    #print(node_)
    source = str(node.source)
    alias = str(node.expression.alias)
    expression = str(node.expression)
    #depth = expression.depth
    
    #full source
    source_list.append(source)
    alias_list.append(alias)
    expression_list.append(expression)
    column_list.append(node_)
    reference_list.append(node.reference_node_name)
    
# Convert to DataFrame
dict = {'output_node': column_list, 'alias': alias_list, 'reference node': reference_list, 'logic': expression_list, 'full source': source_list} 
   
df_i = pd.DataFrame(dict)
df_i

7


,output_node,alias,reference node,logic,full source
0,vendor_ffc,vendor_ffc,,CASE WHEN z.src_sys = 'WD' AND COALESCE(vendor...,SELECT CASE WHEN z.src_sys = 'WD' AND COALESCE...
1,27,src_sys,z,fact_po_hdr.src_sys AS src_sys,SELECT fact_po_hdr.src_sys AS src_sys FROM qma...
2,fact_po_hdr.src_sys,fact_po_hdr,,qma.fact_po_hdr AS fact_po_hdr,qma.fact_po_hdr AS fact_po_hdr
3,27,src_sys,z,fact_po_hdr.src_sys AS src_sys,SELECT fact_po_hdr.src_sys AS src_sys FROM qma...
4,fact_po_hdr.src_sys,fact_po_hdr,,qma.fact_po_hdr AS fact_po_hdr,qma.fact_po_hdr AS fact_po_hdr
5,factory_mapping.vendor_ffc,factory_mapping,,qma_datamart.mapping_ngcv12_factory AS factory...,qma_datamart.mapping_ngcv12_factory AS factory...
6,vendor.vendor_ffc,vendor,,qma.dim_fty_supr AS vendor,qma.dim_fty_supr AS vendor


In [19]:
#join key
df_o.loc[df_o['table']=='vendor']

,output_column,table,action
25,vendor_ffc,vendor,expressions
28,vendor_ffc,vendor,expressions
29,vendor_grp_name,vendor,expressions
38,vendor_grp_name,vendor,expressions
95,changeoperation,vendor,joins
97,fty_code,vendor,joins
99,src_sys,vendor,joins


In [14]:
logics = []

if (token == 'insert'):
    logics.append(df_i.iloc[0]['logic'])
    list_1 = df_i.loc[df_i['output_node'].str.isnumeric()]['logic'].to_list()
else:
    logics.append(df_i.iloc[0]['full source'])
    list_1 = df_i.loc[df_i['output_node'].str.isnumeric()]['full source'].to_list()

logics = logics + list_1
logics

['vendor.vendor_grp_name AS vendor_group_name']

In [15]:
column_f = []
table_f = []

for logic in logics:
    
    column_, table_ = obtain_list_column_table(logic)
    column_f = column_f + column_
    table_f = table_f + table_
    
dict = {'field_name': column_f, 'table_name': table_f} 
df_temp = pd.DataFrame(dict)
df_temp

,field_name,table_name
0,vendor_grp_name,vendor


In [16]:
table_list, table_component, logic_type, original_table, table_schema = obtain_list_table_alias(sql)

# Convert to DataFrame
dict = {'table name': table_list, 'table component': table_component, 'logic': logic_type, 'original table': original_table, 'schema': table_schema} 
   
table_alias = pd.DataFrame(dict)
table_alias

,table name,table component,logic,original table,schema
0,z,"[fact_po_hdr, fact_po_dtl, fact_shp_hdr, fact_...",<class 'sqlglot.expressions.Union'>,,"[qma, qma, qma, qma, qma]"
1,vendor,,,dim_fty_supr,qma
2,factory,,,dim_fty_supr,qma
3,crc,,,mapping_crc_grouping,qma_datamart
4,managing_office,,,sp_managing_office_mapping,qma_datamart
5,factory_mapping,,,mapping_ngcv12_factory,qma_datamart


In [17]:
#clean up
if (token=='insert'):
    df_i['column_l'], df_i['table_l'] = zip(*df_i['logic'].map(obtain_list_column_table))
    df_i.loc[df_i['logic']==df_i['full source'], 'column_l'] = ''
    df_i.loc[df_i['logic']==df_i['full source'], 'table_l'] = ''
elif (token=='update'):
    df_i['column_l'], df_i['table_l'] = zip(*df_i['full source'].map(obtain_list_column_table))
    df_i.loc[df_i['logic']==df_i['full source'], 'column_l'] = ''
    df_i.loc[df_i['logic']==df_i['full source'], 'table_l'] = ''
else:
    print('no action.')
    
df_i

,output_node,alias,reference node,logic,full source,column_l,table_l
0,vendor_group_name,vendor_group_name,,vendor.vendor_grp_name AS vendor_group_name,SELECT vendor.vendor_grp_name AS vendor_group_...,[vendor_grp_name],[vendor]
1,vendor.vendor_grp_name,vendor,,qma.dim_fty_supr AS vendor,qma.dim_fty_supr AS vendor,,


In [131]:
df_mapping = df_i.loc[df_i['output_node']=='']
df_mapping

,output_node,alias,reference node,logic,full source,column_l,table_l


In [132]:
#merge field information to table information
df_field = pd.merge(df_temp, df_mapping, how='left', left_on='table_name', right_on='table')
df_field

KeyError: 'table'

In [109]:
df_i[['output_node', 'logic', 'full source', 'column_l', 'table_l', 'schema', 'table', 'original_table', 'alias', 'field']]
df_j = df_i.loc[df_i['output_node']=='']
df_j = df_j[['schema', 'table', 'table_alias', 'original_table', 'alias', 'field']]

df_z = pd.merge(df_temp, df_j, how='left', left_on = ['field_name', 'table_name'], right_on = ['field', 'table'])
df_z.drop_duplicates(inplace=True)

df_empty_table = df_z.loc[df_z['schema'].isna()]
df_empty_table = pd.merge(df_empty_table[['field_name', 'table_name']], df_j, how='left', left_on = ['field_name'], right_on = ['field'])
df_empty_table.drop_duplicates(inplace=True)

#table field information
df_z = df_z.loc[df_z['schema']=='']
df_final = pd.concat([df_z, df_empty_table], ignore_index=True, axis=0)
df_i.loc[df_i['output_node']!='']

KeyError: "['schema', 'table', 'original_table', 'field'] not in index"

In [ ]:
import sqlglot, string
import sqlglot.expressions as s_ex

def generate_aliases(ast):
   '''
   Take in the SQL AST and generate an infinite stream of possible alias names.
   At each iteration, check if the candidate name does not already exist in the AST
   '''
   ignore = [i.this for i in ast.find_all(s_ex.Identifier)]
   q = ['']
   while q:
     if(current:=q.pop(0)) not in ignore and current: yield current
     for i in string.ascii_lowercase: q += [current + i]

def add_aliases(tree, alias_stream, default_scope = None, scopes = {}):
   def handle_subquery(subquery):
      # function to add alias to subqueries in "from" clause
      if not subquery.alias:
         subquery.args['alias'] = s_ex.TableAlias(this = s_ex.to_identifier(next(alias_stream)))
      add_aliases(subquery.this, alias_stream, default_scope, scopes)
   def handle_table(root):
      # function to add alias to table specified in "from" clause
      scopes[root.this.this] = root.alias if root.alias else next(alias_stream)
      if not root.alias:
          root.args['alias'] = s_ex.TableAlias(this = s_ex.to_identifier(scopes[root.this.this]))
          return root.this.this
   if isinstance(tree, s_ex.Select): # check if tree is a select statement
      if isinstance(root:=tree.args['from'].this, s_ex.Table):
          # if the "from" clause is a table, add alias (if one does not exist)
          default_scope = handle_table(root)
      elif isinstance(root, s_ex.Subquery):
          # if the "from" clause is a subquery, add an alias and traverse it
          handle_subquery(root)
      for i in tree.args.get('joins', []):
          #loop over all table joins, adding aliases and traversing
          if isinstance(i.this, s_ex.Table):
             _ = handle_table(i.this)
          else:
             handle_subquery(i.this)
   if isinstance(tree, s_ex.Column):
      # check if column reference has an alias. If not, add one
      if 'table' not in tree.args:
         tree.args['table'] = s_ex.to_identifier(default_scope if default_scope is not None else next(alias_stream))
      tree.args['table'] = s_ex.to_identifier(scopes.get(tree.args['table'].this, tree.args['table'].this))
   for i, a in enumerate(tree.args.get('expressions', [])):
      if not isinstance(a, sqlglot.expressions.Column) and isinstance(tree, s_ex.Select):
         # check that expression, such as a + b or my_fun(a, b) has an alias
         # if not, add an alias
         tree.args['expressions'][i] = s_ex.Alias(this = a, alias = next(alias_stream))
      add_aliases(a, alias_stream, default_scope, scopes)
   for a, b in getattr(tree, 'args', {}).items():
      # traverse the rest of the tree
      if a not in ['from', 'expressions']:
         for i in ([b] if not isinstance(b, list) else b):
             if not isinstance(i, s_ex.Expression): continue
             add_aliases(i, alias_stream, default_scope, scopes)

In [ ]:
def to_cosette(s):
    ast = sqlglot.parse(s)[0]
    add_aliases(ast, generate_aliases(ast))
    return ast.sql()

s1 = """
 select a, b, c from tbl where a > 1
"""
s2 = """
 select a, count(*) from tbl where my_fun(b + 1) > a + 3 group by a
"""
s3 = """
 select tbl1.a + tbl2.b * 2, from tbl1 join tbl2 on tbl1.c = tbl2.d
"""
s4 = """
 select a + 1 from (select 1 from generate_series(1, 10, 1)) order by a
"""
s5 = """
select name, sum(status in ('. On', '. Off')) from tbl group by name
"""
s6 = """
with cte as (
  select employee_id, starttime, max(endtime) from time_records
  group by employee_id, starttime
)
select employee_id, starttime, m, array_agg(id)
from cte join time_records on cte.starttime <= time_records.starttime and time_records.endtime <= cte.m
where not exists (select 1 from cte where employee_id = employee_id and starttime < starttime and m <= m)
group by employee_id, starttime, m
"""

print(to_cosette(sql))


In [ ]:
sql